In [84]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [85]:
# stats per game
# id: season_start_year, element
player_details = pd.read_csv('artifacts/fetched_data/get_player_details.csv')

In [86]:
# player season summary
# id: season_name, player_name
player_hist = pd.read_csv('artifacts/fetched_data/get_player_hist.csv')

# DON'T USE

In [87]:
# player season summary from a fantasy perspective. Includes element (position)
# id: season_start_year, id
player_info = pd.read_csv('artifacts/fetched_data/get_player_info.csv')

# Use only element. Perhaps in future we could consider if they going to play or not.

# Create data

In [88]:
# fix with data
player_details = player_details.rename(columns={"element":"player_id"})
player_details['kickoff_date'] = pd.to_datetime(player_details['kickoff_time'])
player_details['kickoff_date'] = player_details['kickoff_date'].dt.strftime('%Y-%m-%d')

# add position
player_info = player_info[["season_start_year", "id", "element_type"]].rename(columns={"id":"player_id"})
player = player_details.merge(player_info, on=["season_start_year", "player_id"])
player['position'] = np.where(player.element_type == 1, "G", np.where(player.element_type == 2, "D", np.where(player.element_type == 3, "M", "F")))

# add team
game_list = pd.read_csv('artifacts/fetched_data/get_game_list.csv')
game_list = game_list[["season_start_year", "id", "team_h", "team_a"]].rename(columns={"id":"fixture"})
player = player.merge(game_list, on=["season_start_year", "fixture"])
player['team_id_season'] = np.where(player.was_home == True, player.team_h, player.team_a)
player = player.drop(["team_h", "team_a"], axis=1)

# add next_fixture
from src.components.data.transform.team_form import team_form
teams_fixture = team_form.data_setup()[["season_start_year", "team_id_season", "id", "next_id"]].rename(columns={"id":"fixture"}) # problem when player change team, then next-fixture will be the new team next fixture
player = player.merge(teams_fixture, on=["season_start_year", "team_id_season", "fixture"], how="inner")

# sorting
player.sort_values(["season_start_year", "player_id", "kickoff_date"]).reset_index(drop=True)


,playername,season_start_year,player_id,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,kickoff_date,element_type,position,team_id_season,next_id
0,Shkodran Mustafi,19,1.0,10.0,13.0,0.0,False,2019-08-11T13:00:00Z,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,0.0,33117.0,0.0,0.0,2019-08-11,2,D,1,11.0
1,Shkodran Mustafi,19,1.0,11.0,5.0,0.0,True,2019-08-17T11:30:00Z,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,-5280.0,36709.0,2868.0,8148.0,2019-08-17,2,D,1,24.0
2,Shkodran Mustafi,19,1.0,24.0,10.0,0.0,False,2019-08-24T16:30:00Z,3.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,-6882.0,30975.0,534.0,7416.0,2019-08-24,2,D,1,31.0
3,Shkodran Mustafi,19,1.0,31.0,17.0,0.0,True,2019-09-01T15:30:00Z,2.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,-3872.0,28096.0,346.0,4218.0,2019-09-01,2,D,1,49.0
4,Shkodran Mustafi,19,1.0,49.0,18.0,0.0,False,2019-09-15T15:30:00Z,2.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,-2073.0,26902.0,581.0,2654.0,2019-09-15,2,D,1,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72309,Joseph McGlynn,21,735.0,374.0,14.0,0.0,True,2022-05-22T15:00:00Z,1.0,2.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,1397.0,2242.0,1552.0,155.0,2022-05-22,4,F,5,NaN
72310,Anthony Mancini,21,736.0,367.0,17.0,0.0,False,2022-05-15T11:00:00Z,1.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,2022-05-15,3,M,5,171.0
72311,Anthony Mancini,21,736.0,171.0,2.0,0.0,False,2022-05-19T19:00:00Z,1.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,2022-05-19,3,M,5,374.0
72312,Anthony Mancini,21,736.0,374.0,14.0,0.0,True,2022-05-22T15:00:00Z,1.0,2.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,53.0,73.0,65.0,12.0,2022-05-22,3,M,5,NaN


In [89]:
player.loc[player.playername.str.contains("Frello")]

,playername,season_start_year,player_id,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,kickoff_date,element_type,position,team_id_season,next_id
1146,Jorge Luiz Frello Filho,19,118.0,191.0,1.0,5.0,False,2019-12-29T14:00:00Z,1.0,2.0,20.0,56.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24.0,42.6,3.7,33.0,7.9,49.0,-8456.0,229394.0,2745.0,11201.0,2019-12-29,3,M,6,202.0
1392,Jorge Luiz Frello Filho,19,118.0,238.0,1.0,10.0,True,2020-01-21T20:15:00Z,2.0,2.0,24.0,90.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,36.0,57.6,29.9,53.0,14.1,49.0,1177.0,262966.0,13462.0,12285.0,2020-01-21,3,M,6,244.0
2802,Jorge Luiz Frello Filho,19,118.0,78.0,16.0,1.0,False,2019-10-06T13:00:00Z,1.0,4.0,8.0,90.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,12.0,18.2,3.6,0.0,2.2,50.0,117540.0,374386.0,130332.0,12792.0,2019-10-06,3,M,6,83.0
3395,Jorge Luiz Frello Filho,19,118.0,183.0,16.0,1.0,True,2019-12-26T15:00:00Z,0.0,2.0,19.0,90.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,14.0,24.4,22.4,0.0,4.7,50.0,-9859.0,237315.0,3415.0,13274.0,2019-12-26,3,M,6,191.0
4469,Jorge Luiz Frello Filho,19,118.0,147.0,2.0,1.0,True,2019-12-04T19:30:00Z,2.0,1.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,0.0,0.0,0.2,51.0,-8077.0,296954.0,6752.0,14829.0,2019-12-04,3,M,6,154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60683,Jorge Luiz Frello Filho,21,123.0,342.0,8.0,1.0,False,2022-05-01T13:00:00Z,1.0,0.0,35.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,13.8,2.8,0.0,1.7,57.0,-4396.0,391543.0,4807.0,9203.0,2022-05-01,3,M,6,355.0
60759,Jorge Luiz Frello Filho,21,123.0,355.0,20.0,0.0,True,2022-05-07T14:00:00Z,2.0,2.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,-23834.0,365831.0,1579.0,25413.0,2022-05-07,3,M,6,323.0
60830,Jorge Luiz Frello Filho,21,123.0,323.0,10.0,3.0,False,2022-05-11T18:30:00Z,0.0,3.0,36.0,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,21.0,27.8,0.0,4.9,57.0,-23834.0,365831.0,1579.0,25413.0,2022-05-11,3,M,6,264.0
60899,Jorge Luiz Frello Filho,21,123.0,264.0,9.0,1.0,True,2022-05-19T19:00:00Z,1.0,1.0,37.0,90.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,16.0,21.6,47.3,2.0,7.1,57.0,-7629.0,359531.0,1535.0,9164.0,2022-05-19,3,M,6,375.0


In [93]:
player <- player %>% 
    group_by(player_id) %>%
      arrange(player_id, fixture) %>%
      mutate(
        goals_per_hour = (goals_scored / minutes)*60,
        assists_per_hour = (assists / minutes)*60,
        goals_conceded_per_hour = (goals_conceded / minutes)*60,
        temp = ifelse(minutes < 45, 0, 1),
      ) %>%
      
      mutate(total_goals = rollapply(goals_scored, 38, sum, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             total_assists = rollapply(assists, 38, sum, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             
             mean_goal_5 = rollapply(goals_scored, 5, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_goal_10 = rollapply(goals_scored, 10, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_goal_per_hour_5 = rollapply(goals_per_hour, 5, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_goal_per_hour_10 = rollapply(goals_per_hour, 10, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             
             mean_assists_5 = rollapply(assists, 5, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_assists_10 = rollapply(assists, 10, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_assists_per_hour_5 = rollapply(assists_per_hour, 5, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_assists_per_hour_10 = rollapply(assists_per_hour, 10, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             
             mean_goals_conceded_5 = rollapply(goals_conceded, 5, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_goals_conceded_10 = rollapply(goals_conceded, 10, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_goals_conceded_per_hour_5 = rollapply(goals_conceded_per_hour, 5, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             mean_goals_conceded_per_hour_10 = rollapply(goals_conceded_per_hour, 10, mean, na.rm = TRUE, align = 'right', fill = NA, partial = TRUE),
             
             n_games = rollapply(ifelse(minutes > 45, player_id/player_id, player_id/player_id-1), 38, sum, align = 'right', fill = NA, partial = TRUE),
             minutes_5 = rollapply(minutes, 5, sum, align = 'right', fill = NA, partial = TRUE)
      ) 
      
      team_top <- player %>%
        filter(!is.na(team)) %>%
        group_by(team, fixture, next_fixture) %>%
        summarise(
          n = n(),
          max_value = max(ifelse(is.finite(value), value, NA), na.rm = TRUE),
          max_value_position_g = max(ifelse(position == "G", value, NA), na.rm = TRUE),
          max_value_position_d = max(ifelse(position == "D", value, NA), na.rm = TRUE),
          max_value_position_m = max(ifelse(position == "M", value, NA), na.rm = TRUE),
          max_value_position_f = max(ifelse(position == "F", value, NA), na.rm = TRUE),
          
          max_goal = max(total_goals, na.rm = TRUE),
          max_goal_position_m = max(ifelse(position == "M", total_goals, NA), na.rm = TRUE),
          max_goal_position_f = max(ifelse(position == "F", total_goals, NA), na.rm = TRUE),
          
          max_assists = max(total_assists, na.rm = TRUE),
          max_assists_position_d = max(ifelse(position == "D", total_assists, NA), na.rm = TRUE),
          max_assists_position_m = max(ifelse(position == "M", total_assists, NA), na.rm = TRUE),
          max_assists_position_f = max(ifelse(position == "F", total_assists, NA), na.rm = TRUE)
        ) %>% ungroup() %>%
        # filter out player that switch teams between rounds
        group_by(fixture) %>%  mutate(hierarchy = row_number(desc(n))) %>% ungroup() %>% filter(hierarchy <= 2) %>% select(-hierarchy) %>%
        arrange(team, fixture, next_fixture) 

KeyError: 'goals_per_hour'